In [170]:
import json
import sys
import gzip
from datetime import datetime
import pandas as pd

In [171]:
print("helloo -- this is a test")

helloo -- this is a test


## Extracting Fields

In [172]:
# file = "tweets_work_sample.gz"
file = "1hourtweet.gz"
# file = "nirali3.gz"

with gzip.open(file,'rb') as fin:   
    
    count  = 0
    df = pd.DataFrame(columns=['date', 'tweet_type', 'tweet_text', 
                                   'og_author_id', 'og_author_name', 'og_author_desc', 'og_author_username', 
                                   'tweet_author_id', 'tweet_author_name', 'tweet_author_desc', 'tweet_author_username'])
    for tweet in fin:
        
            
        tweet_d = json.loads(tweet)
#         print(json.dumps(tweet_dict, indent=4))

        created_at = tweet_d['data']['created_at']
    
#         extract date, hour, minute
        date = created_at[0:10]
        time = created_at[11:19]

        new_datetime = datetime.strptime(date+" " +time,'%Y-%m-%d %H:%M:%S')
        
        
        tweet_text = tweet_d['data']['text']
        if 'referenced_tweets' in tweet_d['data']:
            tweet_type = tweet_d['data']['referenced_tweets'][0]['type']
        else:
            tweet_type = None
            og_author_id = None
            og_author_name = None
            og_author_desc = None
            og_author_username = None
            
        
        tweet_author_id = tweet_d['data']['author_id']

        for i in tweet_d['includes']['users']:
            if i['id'] == tweet_author_id:
                tweet_author_id = i['id']
                tweet_author_name = i['name']
                tweet_author_desc = i['description']
                tweet_author_username = i['username']
            elif i['id'] != tweet_author_id:
                og_author_id = i['id']
                og_author_name = i['name']
                og_author_desc = i['description']
                og_author_username = i['username']
          
    
        new_row = pd.DataFrame({'datetime' : new_datetime, 
                        'tweet_type' : tweet_type, 
                        'tweet_text' : tweet_text,
                        'og_author_id' : og_author_id,
                        'og_author_name' : og_author_name,
                        'og_author_desc' : og_author_desc,
                        'og_author_username' : og_author_username,
                        'tweet_author_id' : tweet_author_id,
                        'tweet_author_name' : tweet_author_name,
                        'tweet_author_desc': tweet_author_desc,
                        'tweet_author_username': tweet_author_username
                       },  index=[0])
        df = pd.concat([new_row,df.loc[:]]).reset_index(drop=True)

        
#         stop at 1000
        count += 1
        if count % 100 == 0:
            print(count , "done!")
        if count == 200:
            break
        
        

100 done!
200 done!


In [173]:
df.head(5)

,datetime,tweet_type,tweet_text,og_author_id,og_author_name,og_author_desc,og_author_username,tweet_author_id,tweet_author_name,tweet_author_desc,tweet_author_username,date
0,2022-10-16 04:38:55,retweeted,RT @TexTheLaw: @BenHundeyin That you are unbel...,86379135,SP Benjamin Hundeyin,"@PoliceNG Public Relations Officer, Lagos Stat...",BenHundeyin,2336697621,KingOFRetweets 🇳🇬,"Warri Boy,☺\nEvent Planner ➡C.E.O DessyLomo Ev...",Vj_dessy,NaN
1,2022-10-16 04:38:55,replied_to,@LilliaMarcos it was lining up to go thru the ...,560867723,Lillia Marcos 🐀💉💉💉💉,🇪🇸🇲🇽 made in 🇦🇺 Ancient Tech Writer /Digital P...,LilliaMarcos,2443779228,🦞MUA 4/4 Fully vaxed Union Power,social justice-workers rights-safety net 4the ...,mattmua72,NaN
2,2022-10-16 04:38:55,replied_to,@RM_Transit Tangential to this. I really disli...,1166144223478988800,RMTransit,"Toronto, and Global Rapid Transit Obsessive. D...",RM_Transit,792305478,ansley,3 years car free,perrierappa,NaN
3,2022-10-16 04:38:55,retweeted,RT @ScottLEOWarrior: I will send one random fo...,708584091,Scared To Jump Podcast,The podcast like no other. 🎙 Overcoming menta...,scaredtojumppod,1500498744810565632,penii🐣💜,Tidak ada yang tidak mungkin jika Allah menghe...,queenbigwinn,NaN
4,2022-10-16 04:38:56,retweeted,RT @khalidkarim: Jakarta minimum wage for 48 h...,16995239,khalid karim Presiden Parti Orang Miskin,,khalidkarim,379531765,syæra 🧜🏻‍♀️🪸🌊🫧,I glowin’ in the dark// 💙H,nrsyahirah19,NaN


In [174]:
df.shape

(200, 12)

## Round time to nearest quarter

In [175]:
# 
import datetime
df['rounded_datetime'] = df['datetime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour,15*(dt.minute // 15)))
df.head(5)

,datetime,tweet_type,tweet_text,og_author_id,og_author_name,og_author_desc,og_author_username,tweet_author_id,tweet_author_name,tweet_author_desc,tweet_author_username,date,rounded_datetime
0,2022-10-16 04:38:55,retweeted,RT @TexTheLaw: @BenHundeyin That you are unbel...,86379135,SP Benjamin Hundeyin,"@PoliceNG Public Relations Officer, Lagos Stat...",BenHundeyin,2336697621,KingOFRetweets 🇳🇬,"Warri Boy,☺\nEvent Planner ➡C.E.O DessyLomo Ev...",Vj_dessy,NaN,2022-10-16 04:30:00
1,2022-10-16 04:38:55,replied_to,@LilliaMarcos it was lining up to go thru the ...,560867723,Lillia Marcos 🐀💉💉💉💉,🇪🇸🇲🇽 made in 🇦🇺 Ancient Tech Writer /Digital P...,LilliaMarcos,2443779228,🦞MUA 4/4 Fully vaxed Union Power,social justice-workers rights-safety net 4the ...,mattmua72,NaN,2022-10-16 04:30:00
2,2022-10-16 04:38:55,replied_to,@RM_Transit Tangential to this. I really disli...,1166144223478988800,RMTransit,"Toronto, and Global Rapid Transit Obsessive. D...",RM_Transit,792305478,ansley,3 years car free,perrierappa,NaN,2022-10-16 04:30:00
3,2022-10-16 04:38:55,retweeted,RT @ScottLEOWarrior: I will send one random fo...,708584091,Scared To Jump Podcast,The podcast like no other. 🎙 Overcoming menta...,scaredtojumppod,1500498744810565632,penii🐣💜,Tidak ada yang tidak mungkin jika Allah menghe...,queenbigwinn,NaN,2022-10-16 04:30:00
4,2022-10-16 04:38:56,retweeted,RT @khalidkarim: Jakarta minimum wage for 48 h...,16995239,khalid karim Presiden Parti Orang Miskin,,khalidkarim,379531765,syæra 🧜🏻‍♀️🪸🌊🫧,I glowin’ in the dark// 💙H,nrsyahirah19,NaN,2022-10-16 04:30:00


## Separate Date and Time

In [179]:
for d in df['datetime']:
    df['date'] = d.date()
    df['time'] = d.time()

In [177]:
df.to_csv('1hourtweets_200.csv', index = False)

In [178]:
df.shape

(200, 14)